In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df= pd.read_csv("haifa_trips.csv")
df.columns = ['MAC', 'From', 'To', 'StartTime','EndTime']
df.dropna(inplace = True)
df['MAC'] = pd.to_numeric(df['MAC'])
df['StartTime'] = pd.to_datetime(df['StartTime'])
df['EndTime'] = pd.to_datetime(df['EndTime'])

In [ ]:
df_copy = df.copy()
df_copy.sort_values(['MAC', 'StartTime'],ascending=[True,True], inplace=True)
df_copy = df_copy.reset_index(drop=True)

In [ ]:
ind= df_copy.index.tolist()
length= ind.pop()

#Check the devices have more than one trip - 20 minutes apart from the previous trip
counter=0
for i in range (0, length-1):
    if (df_copy['MAC'].iloc[i]== df_copy['MAC'].iloc[i-1]):
      diff= df_copy['StartTime'].iloc[i]-df_copy['EndTime'].iloc[i-1]
      minutes = diff.total_seconds()/60
      if minutes>20:
        counter=counter+1 
print(counter)


#Check the devices have more than one trip - 10 minutes apart from the previous trip
counter=0
for i in range (0, length-1):
    if (df_copy['MAC'].iloc[i]== df_copy['MAC'].iloc[i-1]):
      diff= df_copy['StartTime'].iloc[i]-df_copy['EndTime'].iloc[i-1]
      minutes = diff.total_seconds()/60
      if minutes>10:
        counter=counter+1 
print(counter)


# The start time of a new section is smaller than the end time of a previous section
counter=0
for i in range (1,length-1):
    if df_copy['MAC'].iloc[i]== df_copy['MAC'].iloc[i-1]:
      diff= df_copy['StartTime'].iloc[i]-df_copy['EndTime'].iloc[i-1]
      minutes = diff.total_seconds()/60
      if minutes < -1:
        counter=counter+1 
print(counter)

#If the segment started and ended in the same minute
counter=0
for i in range (1,length-1):
  if i>0:
      diff= diff= df['StartTime'].iloc[i]-df['EndTime'].iloc[i]
      minutes = diff.total_seconds()/60
      if minutes==0:
        counter=counter+1 
print(counter)


In [ ]:
new_df=df_copy.copy()
len_new_df= new_df.index.tolist()
len= len_new_df.pop()

# Add a new column in the new file and call it the difference
#The difference means time differences between trips of the same person
new_df['Hefresh']= 0
for i in range (1,len):
  if (new_df['MAC'].iloc[i]== new_df['MAC'].iloc[i-1]):
    diff= new_df['StartTime'].iloc[i]-new_df['EndTime'].iloc[i-1]
    minutes = diff.total_seconds()/60
    #new_df['Hefresh'].iloc[i]= minutes
    new_df.iloc[i, new_df.columns.get_loc('Hefresh')] = minutes

new_df['drive_index']=0

x=0
for i in range (1,len):
  if new_df['MAC'].iloc[i]== new_df['MAC'].iloc[i-1] and new_df['Hefresh'].iloc[i]<20:
    #new_df['drive_index'].iloc[i]= x
    new_df.iloc[i, new_df.columns.get_loc('drive_index')] = x
  else:
    x=x+1
    #new_df['drive_index'].iloc[i]= x
    new_df.iloc[i, new_df.columns.get_loc('drive_index')] = x

new_df.iloc[0:50]

In [ ]:
max_drive_index=new_df['drive_index'].iloc[len]
print(max_drive_index)
new_df.tail()

In [ ]:
# Creating a new record that is a trip
max_drive_index=new_df['drive_index'].iloc[len]
trips_arr=[]
trip=["mac","src","dest"]
for i in range (0,max_drive_index):
  rslt_df= new_df.loc[new_df['drive_index'] == i]
  rslt_df_len=rslt_df['MAC'].count()
  trip[0]=rslt_df['MAC'].iloc[0]
  trip[1]=(rslt_df['From'].iloc[0])
  trip[2]=(rslt_df['To'].iloc[rslt_df_len-1])
  trips_arr.append(np.array(trip))

for i in range (0,max_drive_index):
  print(trips_arr[i])

In [ ]:
trips_df = pd.DataFrame(trips_arr, columns = ['MAC', 'src','dest'])
trips_df